In [124]:
# !pip download transformers==4.16.2
# !pip download tokenizers==0.11.0

In [125]:
"""
Libraries
"""
import os
import gc
import ast
import random
import shutil
from pathlib import Path
import itertools
import warnings
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import tokenizers
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig, PreTrainedTokenizerFast
from transformers.models.deberta.tokenization_deberta_fast import DebertaTokenizerFast
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from IPython.core.display_functions import display

In [126]:
"""
tokenizer
"""
# transformers_path = Path("/opt/conda/lib/python3.7/site-packages/transformers")
input_dir = Path("kaggle/input/tokenizers/deberta-v2-3-fast-tokenizer")

transformers_path = Path("/home/noone/anaconda3/envs/NBME/lib/python3.9/site-packages/transformers")

convert_file = input_dir / "convert_slow_tokenizer.py"

conversion_path = transformers_path/convert_file.name

if conversion_path.exists():
    conversion_path.unlink()

shutil.copy(convert_file, transformers_path)
deberta_v2_path = transformers_path / "models" / "deberta_v2"

for filename in ['tokenization_deberta_v2.py', 'tokenization_deberta_v2_fast.py']:
    filepath = deberta_v2_path/filename

    if filepath.exists():
        filepath.unlink()

    shutil.copy(input_dir/filename, filepath)

# CONFIGURATION.tokenizer = PreTrainedTokenizerFast('kaggle/input/checkpoints/deberta-large/tokenizer')


# tokenizer = Tokenizer(BPE())
# tokenizer.pre_tokenizer = Whitespace()
# trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
# tokenizer.train(files=["wiki.train.raw", "wiki.valid.raw", "wiki.test.raw"], trainer=trainer)

In [127]:
"""
Configuration
"""


class CONFIGURATION:
    num_workers = 4
    path = "kaggle/input/checkpoints/deberta-large/"  # todo: adjust this for each scoring attempt
    config_path = path + 'config.pth'
    model = AutoModel.from_pretrained("kaggle/input/models/deberta_large_model")
    batch_size = 24
    fc_dropout = 0.2
    max_len = 466
    seed = 42
    n_fold = 5
    trn_fold = [0, 1, 2, 3, 4]

Some weights of the model checkpoint at kaggle/input/models/deberta_large_model were not used when initializing DebertaModel: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [128]:
"""
settings
"""
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
TOKENIZERS_PARALLELISM = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizers.__version__: 0.10.3
transformers.__version__: 4.14.1


In [129]:
# from transformers.models.deberta_v2.tokenization_deberta_v2_fast import DebertaV2TokenizerFast

tokenizer_file = 'kaggle/input/checkpoints/deberta-large/tokenizer/tokenizer.json'

tokenizer = DebertaTokenizerFast(vocab_file=None,
                     marges_file=None,
                     tokenizer_file=tokenizer_file,
                     bos_token='[CLS]',
                     eos_token='[SEP]',
                     cls_token='[CLS]',
                     unk_token='[UNK]',
                     pad_token='[PAD]',
                     mask_token='[MASK]',
                     add_prefix_space=False)

# tokenizer = DebertaTokenizerFast.from_pretrained('kaggle/input/tokenizers/deberta-tokenizer')
CONFIGURATION.tokenizer = tokenizer

In [130]:
"""
helper functions for scoring
"""
def micro_f1(preds, truths):
    """
    Micro f1 on binary arrays.

    Args:
        preds (list of lists of ints): Predictions.
        truths (list of lists of ints): Ground truths.

    Returns:
        float: f1 score.
    """
    # Micro : aggregating over all instances
    preds = np.concatenate(preds)
    truths = np.concatenate(truths)
    return f1_score(truths, preds)


def spans_to_binary(spans, length=None):
    """
    Converts spans to a binary array indicating whether each character is in the span.
    Args: spans (list of lists of two ints): Spans.
    Returns: np array [length]: Binarized spans.
    """
    length = np.max(spans) if length is None else length
    binary = np.zeros(length)
    for start, end in spans:
        binary[start:end] = 1
    return binary


def span_micro_f1(preds, truths):
    """
    Micro f1 on spans.

    Args:
    preds: list of lists of two ints, prediction spans.
    truths: list of lists of two ints): Ground truth spans.

    Returns: float: f1 score.
    """
    bin_preds = []
    bin_truths = []
    for pred, truth in zip(preds, truths):
        if not len(pred) and not len(truth):
            continue
        length = max(np.max(pred) if len(pred) else 0, np.max(truth) if len(truth) else 0)
        bin_preds.append(spans_to_binary(pred, length))
        bin_truths.append(spans_to_binary(truth, length))
    return micro_f1(bin_preds, bin_truths)


def create_labels_for_scoring(df):
    # example: ['0 1', '3 4'] -> ['0 1; 3 4']
    df['location_for_create_labels'] = [ast.literal_eval(f'[]')] * len(df)
    for i in range(len(df)):
        lst = df.loc[i, 'location']
        if lst:
            new_lst = ';'.join(lst)
            df.loc[i, 'location_for_create_labels'] = ast.literal_eval(f'[["{new_lst}"]]')

    # create labels
    truths = []
    for location_list in df['location_for_create_labels'].values:
        truth = []
        if len(location_list) > 0:
            location = location_list[0]
            for loc in [s.split() for s in location.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                truth.append([start, end])
        truths.append(truth)
    return truths


def get_char_probs(texts, predictions, tokenizer):
    results = [np.zeros(len(t)) for t in texts]
    for i, (text, prediction) in enumerate(zip(texts, predictions)):
        encoded = tokenizer(text,
                            add_special_tokens=True,
                            return_offsets_mapping=True)
        for idx, (offset_mapping, pred) in enumerate(zip(encoded['offset_mapping'], prediction)):
            start = offset_mapping[0]
            end = offset_mapping[1]
            results[i][start:end] = pred
    return results


def get_results(char_probs, th=0.5):
    results = []
    for char_prob in char_probs:
        result = np.where(char_prob >= th)[0] + 1
        result = [list(g) for _, g in itertools.groupby(result, key=lambda n, c=itertools.count(): n - next(c))]
        result = [f"{min(r)} {max(r)}" for r in result]
        result = ";".join(result)
        results.append(result)
    return results


def get_predictions(results):
    predictions = []
    for result in results:
        prediction = []
        if result != "":
            for loc in [s.split() for s in result.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                prediction.append([start, end])
        predictions.append(prediction)
    return predictions

In [131]:
"""
Utils
"""


def get_score(y_true, y_pred):
    score = span_micro_f1(y_true, y_pred)
    return score


def get_logger(filename='inference'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger


LOGGER = get_logger()


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(seed=42)

In [132]:
"""
oof
"""


oof = pd.read_pickle('kaggle/input/checkpoints/deberta-large/oof_df.pkl')

truths = create_labels_for_scoring(oof)
char_probs = get_char_probs(oof['pn_history'].values,
                            oof[[i for i in range(CONFIGURATION.max_len)]].values,
                            CONFIGURATION.tokenizer)
best_th = 0.5
best_score = 0.
for th in np.arange(0.45, 0.55, 0.01):
    th = np.round(th, 2)
    results = get_results(char_probs, th=th)
    preds = get_predictions(results)
    score = get_score(preds, truths)
    if best_score < score:
        best_th = th
        best_score = score
    LOGGER.info(f"th: {th}  score: {score:.5f}")
LOGGER.info(f"best_th: {best_th}  score: {best_score:.5f}")

th: 0.45  score: 0.37183
th: 0.45  score: 0.37183
th: 0.45  score: 0.37183
th: 0.45  score: 0.37183
th: 0.45  score: 0.37183
th: 0.45  score: 0.37183
th: 0.45  score: 0.37183
th: 0.45  score: 0.37183
th: 0.45  score: 0.37183
th: 0.46  score: 0.37170
th: 0.46  score: 0.37170
th: 0.46  score: 0.37170
th: 0.46  score: 0.37170
th: 0.46  score: 0.37170
th: 0.46  score: 0.37170
th: 0.46  score: 0.37170
th: 0.46  score: 0.37170
th: 0.46  score: 0.37170
th: 0.47  score: 0.37164
th: 0.47  score: 0.37164
th: 0.47  score: 0.37164
th: 0.47  score: 0.37164
th: 0.47  score: 0.37164
th: 0.47  score: 0.37164
th: 0.47  score: 0.37164
th: 0.47  score: 0.37164
th: 0.47  score: 0.37164
th: 0.48  score: 0.37138
th: 0.48  score: 0.37138
th: 0.48  score: 0.37138
th: 0.48  score: 0.37138
th: 0.48  score: 0.37138
th: 0.48  score: 0.37138
th: 0.48  score: 0.37138
th: 0.48  score: 0.37138
th: 0.48  score: 0.37138
th: 0.49  score: 0.37128
th: 0.49  score: 0.37128
th: 0.49  score: 0.37128
th: 0.49  score: 0.37128


In [133]:
"""
Data Loading
"""


test = pd.read_csv('kaggle/input/nbme-score-clinical-patient-notes/test.csv')
submission = pd.read_csv('kaggle/input/nbme-score-clinical-patient-notes/sample_submission.csv')
features = pd.read_csv('kaggle/input/nbme-score-clinical-patient-notes/features.csv')


def preprocess_features(features):
    features.loc[27, 'feature_text'] = "Last-Pap-smear-1-year-ago"
    return features


features = preprocess_features(features)
patient_notes = pd.read_csv('kaggle/input/nbme-score-clinical-patient-notes/patient_notes.csv')
print(f"test.shape: {test.shape}")
display(test.head())
print(f"features.shape: {features.shape}")
display(features.head())
print(f"patient_notes.shape: {patient_notes.shape}")
display(test.head())
test = test.merge(features, on=['feature_num', 'case_num'], how='left')
test = test.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(test.head())

test.shape: (5, 4)


,id,case_num,pn_num,feature_num
0,00016_000,0,16,0
1,00016_001,0,16,1
2,00016_002,0,16,2
3,00016_003,0,16,3
4,00016_004,0,16,4


features.shape: (143, 3)


,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure
3,3,0,Intermittent-symptoms
4,4,0,Lightheaded


patient_notes.shape: (42146, 3)


,id,case_num,pn_num,feature_num
0,00016_000,0,16,0
1,00016_001,0,16,1
2,00016_002,0,16,2
3,00016_003,0,16,3
4,00016_004,0,16,4


,id,case_num,pn_num,feature_num,feature_text,pn_history
0,00016_000,0,16,0,Family-history-of-MI-OR-Family-history-of-myoc...,HPI: 17yo M presents with palpitations. Patien...
1,00016_001,0,16,1,Family-history-of-thyroid-disorder,HPI: 17yo M presents with palpitations. Patien...
2,00016_002,0,16,2,Chest-pressure,HPI: 17yo M presents with palpitations. Patien...
3,00016_003,0,16,3,Intermittent-symptoms,HPI: 17yo M presents with palpitations. Patien...
4,00016_004,0,16,4,Lightheaded,HPI: 17yo M presents with palpitations. Patien...


In [134]:
"""
Dataset
"""


def prepare_input(cfg, text, feature_text):
    inputs = cfg.tokenizer(text, feature_text,
                           add_special_tokens=True,
                           max_length=CONFIGURATION.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.feature_texts = df['feature_text'].values
        self.pn_historys = df['pn_history'].values

    def __len__(self):
        return len(self.feature_texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg,
                               self.pn_historys[item],
                               self.feature_texts[item])
        return inputs

In [135]:
"""
Model
"""


class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, 1)
        self._init_weights(self.fc)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        return last_hidden_states

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

In [136]:
"""
inference
"""


def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions


test_dataset = TestDataset(CONFIGURATION, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CONFIGURATION.batch_size,
                         shuffle=False,
                         num_workers=CONFIGURATION.num_workers, pin_memory=True, drop_last=False)
predictions = []

for fold in CONFIGURATION.trn_fold:
    model = CustomModel(CONFIGURATION, config_path=CONFIGURATION.config_path, pretrained=False)
    state = torch.load(CONFIGURATION.path + f"{CONFIGURATION.model.split('/')[1]}_fold{fold}_best.pth", map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    prediction = prediction.reshape((len(test), CONFIGURATION.max_len))
    char_probs = get_char_probs(test['pn_history'].values, prediction, CONFIGURATION.tokenizer)
    predictions.append(char_probs)
    del model, state, prediction, char_probs; gc.collect()
    torch.cuda.empty_cache()
predictions = np.mean(predictions, axis=0)

AttributeError: 'DebertaModel' object has no attribute 'split'

In [ ]:
"""
submission
"""


results = get_results(predictions, th=best_th)
submission['location'] = results
display(submission.head())
submission[['id', 'location']].to_csv('submission.csv', index=False)